# Functional analysis of single-cell transcriptomics

## Introduction

Single-cell transcriptomics yield many molecular readouts that one by one are
hard to interpret. Functional analysis or "enrichment" tries to summarize this
information into interpretable concepts using prior knowledge, making it easier
to characterize the biological context being studied. 

The goals of this tutorial are to:

- Introduce the functional analysis concept
- Learn how to run functional analysis
- Understand the differences in functional modeling
- Interpret the obtained scores

To do so, we will analyse a downsampled single-cell transcriptomics dataset
consisting of human immune cells from healthy and COVID-19 infected patients.
We leverage the tool [decoupler](https://decoupler-py.readthedocs.io/en/latest/), which contains enrichment methods
to obtain biological activities from omics data.

## Dependencies
To be able to run the code, we first need to load
the required packages.

In [ ]:
# Single-cell processing
import scanpy as sc

# Enrichment analysis
import decoupler as dc

# Data handling
import pandas as pd
import numpy as np

# Plotting options, change to your liking
sc.settings.set_figure_params(dpi=200, frameon=False)
sc.set_figure_params(dpi=200)
sc.set_figure_params(figsize=(4, 4))

## Data loading
To load the processed dataset run. With this we obtain an `AnnData` object:

In [ ]:
adata = sc.read_h5ad('data/covid.h5ad')
adata



Inside an `AnnData` object we have different attributes, here are the most important ones:

* X: This is where the main assay is stored, in this case log-transformed normalized counts
* obs: Metadata dataframe for cells (rows)
* var: Metadata dataframe for genes (columns)

You can read more about it in the official [documentation](https://anndata.readthedocs.io/en/latest/index.html)

<img src="https://anndata.readthedocs.io/en/latest/_images/anndata_schema.svg"
     width="400" 
     height="500" />

What can you tell about this already processed dataset? Explore its metadata and
try to plot some of its attributes with `sc.pl.umap`:

<div class="alert alert-info">

**Note**
    
If you want to know how to use a function, you can run `?` followed with the function name. Example `?sc.pl.umap`.

</div>  

In [ ]:
# Explore adata




## Cell type annotation

In single-cell data, we have no prior information of which cell type each cell
belongs. To assign cell type labels, we first project all cells in a shared
embedded space, then we find communities of cells that show a similar
transcription profile and finally we check what cell type specific markers are
expressed. These genes are mainly expressed exclusively by a specific
cell type, making them useful to distinguish heterogeneous groups of cells.
Marker genes were discovered and annotated in previous studies and there are
some resources that collect and curate them. If more than one marker gene is
available, statistical methods can be used to test if a set of markers is
enriched in a given cell population.

`PanglaoDB` is a database of cell type markers, which can be easily accessed
using a wrapper to `OmniPath` from `decoupler`:

```python
# Retrieving via decoupler
markers = dc.get_resource('PanglaoDB')
```

However, for stability of this tutorial we are using a fixed version:

In [ ]:
markers = pd.read_csv('data/panglaodb.csv')
markers

Take a look how the resource is organized. Can you filter by relevant 
information? Think of a filtering strategy that will help the annotation of your
dataset.

In [ ]:
# Filter relevant information




Once filtered, we will use the resource `PanglaoDB` and the method Over Representation Analysis (ORA) to see which cells are
enriched by cell type marker genes.

In [ ]:
dc.run_ora(
    mat=adata,
    net=markers,
    source='cell_type',
    target='genesymbol',
    min_n=3,
    verbose=True,
    use_raw=False
)

For each cell, we now have an enrichment score for each cell type in our resource. We can extract these scores in a new `AnnData` object:

In [ ]:
acts = dc.get_acts(adata, obsm_key='ora_estimate')

# We need to remove inf and set them to the maximum value observed
acts_v = acts.X.ravel()
max_e = np.nanmax(acts_v[np.isfinite(acts_v)])
acts.X[~np.isfinite(acts.X)] = max_e

# We can scale the obtained activities for better visualizations
sc.pp.scale(acts)
acts

With `decoupler` we can also identify which are the top predicted cell types per cluster using the 
function `dc.rank_sources_groups`. Here, it identifies "marker" cell types per cluster using
same statistical tests available in scanpy's `scanpy.tl.rank_genes_groups`.

In [ ]:
# Extract top cell types per cluster
df = dc.rank_sources_groups(acts, groupby='leiden', reference='rest', method='t-test_overestim_var')
df

We can then extract the top 3 predicted cell types per cluster:

In [ ]:
n_ctypes = 3
ctypes_dict = df.groupby('group').head(n_ctypes).groupby('group')['names'].apply(lambda x: list(x)).to_dict()
ctypes_dict

We can visualize the obtained top predicted cell types:

In [ ]:
sc.pl.matrixplot(acts, ctypes_dict, 'leiden', dendrogram=True,
                 colorbar_title='Z-scaled scores', vmin=-2, vmax=2, cmap='RdBu_r')

Can you come up with an annotation for this atlas? You can use `sc.pl.violin` and `sc.pl.umap` to explore the obtained scores.

In [ ]:
# Explore data




In [ ]:
# Final anntoation
ann_dict = {
    '0': '',
    '1': '',
    '2': '',
    '3': '',
    '4': '',
    '5': '',
    '6': '',
    '7': '',
}

adata.obs['cell_type'] = [ann_dict[clust] for clust in adata.obs['leiden']]

We can visualize how the final annotation looks like:

In [ ]:
sc.pl.umap(adata, color='cell_type')

## Transcription factor activity inference
Transcription factors (TF) are proteins that regulate the expression of specific 
target genes. Gene regulation can be represented as a Gene Regulatory Network
(GRN), where TFs regulate downstream target genes (which can be other TFs). 

[DoRothEA](https://saezlab.github.io/dorothea/) is a comprehensive resource
containing a curated collection of TFs and their transcriptional targets. Since
these regulons were gathered from different types of evidence, interactions in
DoRothEA are classified in different confidence levels, ranging from A (highest
confidence) to D (lowest confidence). Moreover, each interaction is weighted by
its confidence level and the sign of its mode of regulation (activation or
inhibition).

It can be easily accessed using a wrapper to `OmniPath` from `decoupleR`.

```python
# Retrieving via decoupler
dorothea = dc.get_dorothea()
```

However, like in the previous example we will work with a fixed version:

In [ ]:
dorothea = pd.read_csv('data/dorothea.csv')
dorothea

Explore a little bit the obtained network. How many interactions are coming
from each confidence level? What is the distribution of mode of regulations?
What is the distribution of number of target genes per TF?

In [ ]:
# Explore dorothea




Afterwards, we will give the gene expression and `DoRothEA` to
the method univariate linear model (ULM), which will infer activity scores for each TF
in each cell.

In [ ]:
dc.run_ulm(
    mat=adata,
    net=dorothea,
    source='source',
    target='target',
    weight='weight',
    min_n=10,
    verbose=True,
    use_raw=False
)

For each cell, we now have an enrichment score for each TF in our network. We can extract these scores in a new `AnnData` object:

In [ ]:
acts = dc.get_acts(adata, obsm_key='ulm_estimate')

# We can scale the obtained activities for better visualizations
sc.pp.scale(acts)
acts

Like before, we can test which are the top active TFs per cluster or cell type:

In [ ]:
# Extract top cell types per cluster
df = dc.rank_sources_groups(acts, groupby='cell_type', reference='rest', method='t-test_overestim_var')
df

Explore the results with `sc.pl.matrixplot`, `sc.pl.violin` and `sc.pl.umap`. Do these results make sense? Search some active TFs in the literature to
confirm.

# Pathway activity inference

Cells activate or inhibit gene programs, called pathways, that module the cell
response to external changes. For example they can induce apoptosis if they are
targeted by immune cells or shift metabolism when oxygen is not available.

[PROGENy](https://saezlab.github.io/progeny/) is a comprehensive resource
containing a curated collection of pathways and their target genes, with
weights (activation or inhibition) and p-values for each interaction.

It can be easily accessed using a wrapper to `OmniPath` from `decoupleR`.

```python
progeny = dc.get_progeny()
```

But we will work with a fixed version:

In [ ]:
progeny = pd.read_csv('data/progeny.csv')
progeny

Explore the obtained dataframe. How many different pathways does PROGENy
contain. Discuss some of its pathways, what biological processes are they
regulating? Search literature if needed.

In the previous exercise, we estimated TF activities per cell type but we did
not explore the differences between healthy and COVID-19 patients. To do so,
first we will compute the Differential Expressed Genes (DEG) between conditions for a cell type of your choice.

In [ ]:
# Select your cell type of choice
cell_type = ''

# Subset adata
sdata = adata[adata.obs['cell_type'] == cell_type]

# Run DEG between COVID and healthy
sdata.uns['log1p']["base"] = None # Might need this
sc.tl.rank_genes_groups(sdata, 'disease', method='t-test')

# Extract DEG
deg = sc.get.rank_genes_groups_df(sdata, group=None)
deg.head(10)

What can you tell about the top DEG? Try looking at literature for meaning.

Results from DEG can be used for functional enrichment. Now we will use the
obtained logFCs and estimate pathway activities from them using ULM:

In [ ]:
# Filter sign genes and transform to mat
mat = deg[deg['pvals_adj'] < 0.05].pivot(index='group', columns='names', values='logfoldchanges')
mat

In [ ]:
enr_scores, enr_pvals = dc.run_ulm(
    mat=mat,
    net=progeny,
    source='source',
    target='target',
    weight='weight',
    min_n=10,
    verbose=True,
    use_raw=False
)

enr_scores

We can visualize the results:

In [ ]:
dc.plot_barplot(enr_scores, 'COVID-19', top=25, vertical=False)

Comment the obtained results. Do they make sense? Explore other cell types.